In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, asc, desc, avg, count, when, year, to_date, lit, max, min, row_number,format_number,aggregate,sum
from pyspark.sql.window import Window
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# load the first dataset about the fifa players (change the 'paths.csv')
df = pd.read_csv('data/raw/male_players.csv', low_memory=False)
# transform the dataset into parquet file 
df.to_parquet('data/processed/fifa_player.parquet', engine='pyarrow', compression='snappy')
if 'fifa_player.parquet' in os.listdir('data/processed'):
    print("Conversion complete.")

Conversion complete.


In [3]:
# load the second dataset about teams
df = pd.read_csv('data/raw/male_teams.csv', low_memory = False)
# transform the dataset into parquet file
df.to_parquet('data/processed/teams.parquet' , engine = 'pyarrow', compression = 'snappy')
if 'teams.parquet' in os.listdir('data/processed'):
    print("Conversion complete.")

Conversion complete.


In [4]:
# load dataset about coaches
df = pd.read_csv('data/raw/male_coaches.csv', low_memory = False)
# transform into parquet
df.to_parquet('data/processed/coaches.parquet' , engine = 'pyarrow', compression = 'snappy')
if 'coaches.parquet' in os.listdir('data/processed'):
    print("Conversion complete.")

Conversion complete.


In [5]:
# start the spark session
spark = SparkSession.builder \
    .appName("FIFA_Analysis") \
    .getOrCreate()

# read the file as parquet
df = spark.read.parquet("data/processed/fifa_player.parquet")
print(f"Player rows loaded: {df.count()}")

Player rows loaded: 180021


In [6]:
# read the parquet files (previously converted) 
df_players = spark.read.parquet("data/processed/fifa_player.parquet")
df_teams = spark.read.parquet("data/processed/teams.parquet")
df_coaches = spark.read.parquet("data/processed/coaches.parquet")

In [7]:
# transform the columns into integers
df_players = df_players.withColumn("fifa_version", col("fifa_version").cast("int"))
df_players = df_players.withColumn("value_eur", col("value_eur").cast("int"))
df_teams = df_teams.withColumn("transfer_budget_eur", col("transfer_budget_eur").cast("int"))
df_players = df_players.withColumn("wage_eur", col("wage_eur").cast("int"))

In [8]:
# from here on we performed some exploratory checks to assess and test our analysis
# find potential keys in dataframe
def find_keys(df, filename):
    cols = df.columns
    # search keys in column for merge
    keys = [c for c in cols if "id" in c or "team" in c or "coach" in c]
    print(f"--- Possible keys in {filename} ---")
    print(keys)
    print("-" * 50)

# search keys
find_keys(df_players, "PLAYERS")
find_keys(df_teams, "TEAMS")
find_keys(df_coaches, "COACHES")

--- Possible keys in PLAYERS ---
['player_id', 'club_team_id', 'league_id', 'club_contract_valid_until_year', 'nationality_id', 'nation_team_id', 'defending_sliding_tackle']
--------------------------------------------------
--- Possible keys in TEAMS ---
['team_id', 'team_url', 'team_name', 'league_id', 'nationality_id', 'midfield', 'coach_id', 'rival_team', 'whole_team_average_age', 'def_team_width', 'def_team_depth', 'def_defence_width', 'off_team_width']
--------------------------------------------------
--- Possible keys in COACHES ---
['coach_id', 'coach_url', 'coach_face_url']
--------------------------------------------------


In [9]:
# merge dataframes to check total raws of players after join
df_teams_full = df_teams.join(
    df_coaches, 
    on="coach_id", 
    how="left"
)

df_final = df_players.join(
    df_teams_full,
    df_players["club_team_id"] == df_teams_full["team_id"],
    how="left"
)

print(f"Total rows (spurious): {df_final.count()}")
# total number of player printed this way is wrong
# a mega join is useless for our analysis 

Total rows (spurious): 1417641


In [10]:
#df structure
df_players.printSchema()
df_teams.printSchema()
df_coaches.printSchema()

root
 |-- player_id: long (nullable = true)
 |-- player_url: string (nullable = true)
 |-- fifa_version: integer (nullable = true)
 |-- fifa_update: double (nullable = true)
 |-- update_as_of: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: long (nullable = true)
 |-- potential: long (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- wage_eur: integer (nullable = true)
 |-- age: long (nullable = true)
 |-- dob: string (nullable = true)
 |-- height_cm: long (nullable = true)
 |-- weight_kg: long (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_id: double (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: double (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: double (nullable = true)
 |-- club_loaned_from: string (nullabl

In [11]:
##### QUERIES #####

In [12]:
# QUERY 1 - MOST PAYED PLAYER FOR EACH FIFA VERSION
window_spec = Window.partitionBy("fifa_version").orderBy(desc("wage_eur"))
df_players.withColumn("rank", row_number().over(window_spec)) \
  .filter("rank = 1") \
  .orderBy("fifa_version") \
  .select(col("short_name").alias("Player"), "age", "club_name", col("wage_eur").alias("Weekly wage (euro)"), 'fifa_version') \
  .show()

+-----------------+---+---------------+------------------+------------+
|           Player|age|      club_name|Weekly wage (euro)|fifa_version|
+-----------------+---+---------------+------------------+------------+
|         L. Messi| 27|   FC Barcelona|            550000|          15|
|         L. Messi| 28|   FC Barcelona|            550000|          16|
|Cristiano Ronaldo| 31|    Real Madrid|            575000|          17|
|Cristiano Ronaldo| 32|    Real Madrid|            575000|          18|
|         L. Messi| 31|   FC Barcelona|            575000|          19|
|         L. Messi| 32|   FC Barcelona|            560000|          20|
|         L. Messi| 33|   FC Barcelona|            560000|          21|
|     K. De Bruyne| 30|Manchester City|            350000|          22|
|       K. Benzema| 34|    Real Madrid|            450000|          23|
|     K. De Bruyne| 32|Manchester City|            350000|          24|
+-----------------+---+---------------+------------------+------

In [13]:
# QUERY 2 - LIVERPOOL PLAYERS FOR FIFA VERSION 2020
df_players.select("short_name", "age" , "club_name" , "fifa_version" , "overall", "potential", "preferred_foot","player_positions")\
.filter(df_players['club_name'] == 'Liverpool')\
.filter(df_players['fifa_version'] == 20)\
.show(33)

# we display 33 rows at it is the number of Liverpool players; we obtained it by substituting the .show() function with .count()

+--------------------+---+---------+------------+-------+---------+--------------+----------------+
|          short_name|age|club_name|fifa_version|overall|potential|preferred_foot|player_positions|
+--------------------+---+---------+------------+-------+---------+--------------+----------------+
|         V. van Dijk| 27|Liverpool|          20|     90|       91|         Right|              CB|
|            M. Salah| 27|Liverpool|          20|     90|       90|          Left|          RW, ST|
|             Alisson| 26|Liverpool|          20|     89|       91|         Right|              GK|
|             S. Mané| 27|Liverpool|          20|     88|       88|         Right|          LW, LM|
|     Roberto Firmino| 27|Liverpool|          20|     86|       87|         Right|     CF, ST, CAM|
|             Fabinho| 25|Liverpool|          20|     85|       89|         Right|             CDM|
|        A. Robertson| 25|Liverpool|          20|     85|       89|          Left|              LB|


In [14]:
# 1. distinct players dataframe
df_distinct = df_players.dropDuplicates(["short_name"])
# 2. number of distinct players
total_distinct_players = df_distinct.count()
print(f"Total DISTINCT Players: {total_distinct_players}")

Total DISTINCT Players: 47272


In [15]:
# QUERY 3 - FOOT DISTRIBUTION IN THE DATABASE
# 3. calculation of percentages
df_result_d = df_distinct.groupBy("preferred_foot") \
    .agg(count("*").alias(" Number of players ")) \
    .withColumn("Percentage", (col(" Number of players ") / lit(total_distinct_players)) * 100) \
    .orderBy(col(" Number of players ").desc()) \
    .select( col("preferred_foot").alias(" Preferred foot "), " Number of players ", format_number(col("Percentage"), 2).alias(" % on total"))
df_result_d.show()

+----------------+-------------------+-----------+
| Preferred foot | Number of players | % on total|
+----------------+-------------------+-----------+
|           Right|              36369|      76.94|
|            Left|              10903|      23.06|
+----------------+-------------------+-----------+



In [16]:
df_players = df_players.withColumn("player_name", col("short_name"))
df_coaches = df_coaches.withColumn("coach_name", col("short_name"))

In [17]:
df_team_coach = df_coaches.join(df_teams, on=["coach_id"],how = "inner")
#df_team_coach.printSchema()
df_team_coach = df_team_coach.withColumn("fifa_version", col("fifa_version").cast("int"))

In [18]:
# I. SETUP: Create intermediate DataFrame joining Teams and Coaches
# This intermediate DataFrame contains the year (fifa_version) taken from Teams
df_team_coach = df_teams.join(
    df_coaches, 
    on="coach_id", 
    how="inner"
)

In [19]:
target_player = "Cristiano Ronaldo" 

# QUERY 4 - HISTORY OF COACHES FOR TARGET PLAYER (CR)
df_history = df_players.filter(col("player_name") == target_player) \
    .join(
        df_team_coach,
        (df_players.club_team_id == df_team_coach.team_id) & 
        (df_players.fifa_version == df_team_coach.fifa_version), 
        "inner"
    ) \
    .select(
        col("player_name").alias("Player"),
        df_players.fifa_version.alias("FIFA_Edition"), 
        col("club_name").alias("Team"),
        df_coaches.short_name.alias("Coach") 
    ) \
    .dropDuplicates(["FIFA_Edition"]) \
    .orderBy("FIFA_Edition").show()

+-----------------+------------+-----------------+--------------------+
|           Player|FIFA_Edition|             Team|               Coach|
+-----------------+------------+-----------------+--------------------+
|Cristiano Ronaldo|          15|      Real Madrid|        C. Ancelotti|
|Cristiano Ronaldo|          16|      Real Madrid|        C. Ancelotti|
|Cristiano Ronaldo|          17|      Real Madrid|        C. Ancelotti|
|Cristiano Ronaldo|          18|      Real Madrid|        C. Ancelotti|
|Cristiano Ronaldo|          19|         Juventus|          M. Allegri|
|Cristiano Ronaldo|          20|         Juventus|          M. Allegri|
|Cristiano Ronaldo|          21|         Juventus|          M. Allegri|
|Cristiano Ronaldo|          22|Manchester United|          E. ten Hag|
|Cristiano Ronaldo|          23|Manchester United|          E. ten Hag|
|Cristiano Ronaldo|          24|         Al Nassr|L. Ribeiro de Castro|
+-----------------+------------+-----------------+--------------

In [20]:
# QUERY 5 - SERIE B TEAMS RANKING FOR FIFA VERSION 20
target_version = 20


p = df_players.alias("p")
t = df_teams.alias("t")

df_serie_b_ranking = (
    p.filter(col("p.fifa_version") == target_version)
    .join(
        t.filter(
            (col("t.fifa_version") == target_version) &
            (col("t.league_name") == "Serie B")
        ),
        col("p.club_team_id") == col("t.team_id"),
        "inner"
    )
    .groupBy(col("p.club_name").alias("club_name"))
    .agg(
        count("*").alias("Num_players"),
        format_number(avg(col("p.overall")), 2).alias("Avg_overall"),
        sum(col("p.value_eur").cast("long")).alias("Total_value")
    )
    .orderBy(desc("Total_value"))
)
df_serie_b_ranking.show(20, truncate=False)

+-----------------------+-----------+-----------+-----------+
|club_name              |Num_players|Avg_overall|Total_value|
+-----------------------+-----------+-----------+-----------+
|Sampdoria              |32         |70.62      |141560000  |
|Parma                  |33         |69.73      |95120000   |
|Brescia                |28         |69.57      |90625000   |
|SPAL                   |27         |69.78      |85945000   |
|Chievo                 |30         |65.77      |44890000   |
|Benevento              |23         |68.61      |39770000   |
|Perugia                |26         |66.42      |32905000   |
|Pescara                |30         |64.27      |25545000   |
|Crotone                |26         |63.81      |25250000   |
|Spezia                 |30         |64.07      |24355000   |
|Cremonese              |24         |66.46      |22470000   |
|Ascoli                 |29         |63.72      |22440000   |
|Cittadella             |26         |65.50      |21995000   |
|Pisa   

In [21]:
# QUERY 6 - TALL AND HEAVY PLAYERS (GIANTS)
df_giants_unique = df_players.filter((col("height_cm") > 190) & (col("weight_kg") > 90)) \
    .select("player_name", "height_cm", "weight_kg", "club_name", "player_positions",'age','fifa_version') \
    .dropDuplicates(["player_name"]) \
    .orderBy(desc("height_cm"))
df_giants_unique.show(5)

+-----------+---------+---------+---------------+----------------+---+------------+
|player_name|height_cm|weight_kg|      club_name|player_positions|age|fifa_version|
+-----------+---------+---------+---------------+----------------+---+------------+
|K. Van Hout|      207|      110|       Westerlo|              GK| 29|          17|
|K. Gadellaa|      206|       92|     FC Utrecht|              GK| 20|          24|
|   I. Touré|      206|       98|        Lorient|              CB| 20|          24|
|    T. Holý|      206|      102|Carlisle United|              GK| 31|          24|
|        Idé|      204|       96|       Boavista|              ST| 22|          17|
+-----------+---------+---------+---------------+----------------+---+------------+
only showing top 5 rows


In [22]:
# Register the DataFrame as a temporary view
df_players.createOrReplaceTempView("fifa_players")

# QUERY 7 - ITALIAN PLAYERS BEST VERSIONS
'''Advanced SQL Query (Deduplication with Window Function)
   This query finds the top 10 Italian players based on their best overall rating across all FIFA versions
   It uses ROW_NUMBER() to deduplicate players by keeping only their strongest version'''

query_sql = """
    SELECT player_name, club_name, overall, fifa_version
    FROM (
        SELECT 
            player_name, 
            club_name, 
            overall, 
            fifa_version,
            -- Assign a rank to each player version (1 = highest overall rating)
            ROW_NUMBER() OVER (
                PARTITION BY player_name 
                ORDER BY overall DESC
            ) as internal_ranking
        FROM fifa_players
        WHERE nationality_name = 'Italy'
    ) 
    -- Keep only the top-ranked version for each player
    WHERE internal_ranking = 1 AND fifa_version = 20
    ORDER BY overall DESC
    LIMIT 20
"""
spark.sql(query_sql).show(truncate=False)

+---------------+-----------------------+-------+------------+
|player_name    |club_name              |overall|fifa_version|
+---------------+-----------------------+-------+------------+
|G. Chiellini   |Juventus               |89     |20          |
|F. Bernardeschi|Juventus               |83     |20          |
|F. Quagliarella|Sampdoria              |83     |20          |
|D. Rugani      |Juventus               |82     |20          |
|M. Politano    |Inter                  |82     |20          |
|S. El Shaarawy |Shanghai Shenhua       |82     |20          |
|A. Izzo        |Torino                 |81     |20          |
|M. Caldara     |Milan                  |80     |20          |
|D. Baselli     |Torino                 |79     |20          |
|L. Pavoletti   |Cagliari               |78     |20          |
|P. Cutrone     |Wolverhampton Wanderers|77     |20          |
|K. Lasagna     |Udinese                |77     |20          |
|Rômulo         |Genoa                  |77     |20    

In [23]:
# Re-register temporary views in case the Spark session was restarted
df_players.createOrReplaceTempView("players")
df_teams.createOrReplaceTempView("teams")
df_coaches.createOrReplaceTempView("coaches")

# QUERY 8 - NATIONAL ELITE SYNERGY
''': Find top players (overall > 80) who share the same nationality as their coach
    - Join players to teams on club_team_id and matching fifa_version to ensure same edition
    - Join teams to coaches via coach_id
    - Filter to FIFA version 22 and nationality match between player and coach
    - Order by player overall descending and limit to top 15 results'''  
query_compatriots = """
    SELECT 
        p.short_name AS Player,
        c.short_name AS Coach,
        p.nationality_name AS Nation,
        t.team_name AS Team,  -- Use team_name from teams
        p.overall
    FROM players p
    JOIN teams t 
        ON p.club_team_id = t.team_id 
        AND p.fifa_version = t.fifa_version
    JOIN coaches c 
        ON t.coach_id = c.coach_id
    WHERE 
        p.fifa_version = 22
        AND p.nationality_name = c.nationality_name 
        AND p.overall > 80
    ORDER BY p.overall DESC
    LIMIT 15
"""

# Execute the query and display results without truncating strings
spark.sql(query_compatriots).show(truncate=False)

+---------------+-----------------------+-------+-------------------+-------+
|Player         |Coach                  |Nation |Team               |overall|
+---------------+-----------------------+-------+-------------------+-------+
|K. Mbappé      |C. Galtier             |France |Paris Saint Germain|91     |
|M. Neuer       |J. Nagelsmann          |Germany|FC Bayern München  |90     |
|J. Kimmich     |J. Nagelsmann          |Germany|FC Bayern München  |89     |
|L. Goretzka    |J. Nagelsmann          |Germany|FC Bayern München  |87     |
|T. Müller      |J. Nagelsmann          |Germany|FC Bayern München  |87     |
|C. Immobile    |M. Sarri               |Italy  |Lazio              |87     |
|A. Laporte     |J. Guardiola i Sala    |Spain  |Manchester City    |86     |
|Rodri          |J. Guardiola i Sala    |Spain  |Manchester City    |86     |
|G. Chiellini   |M. Allegri             |Italy  |Juventus           |86     |
|Jordi Alba     |X. Hernández           |Spain  |FC Barcelona   

In [24]:
# --- QUERY 9 - BRAZILIAN PLAYERS IN PREMIER LEAGUE ---
query_brazilians = """
    SELECT 
        p.short_name AS Player,
        p.player_positions AS Positions,
        t.team_name AS Team,
        p.overall
    FROM players p
    JOIN teams t 
        ON p.club_team_id = t.team_id 
        AND p.fifa_version = t.fifa_version
    WHERE 
        p.fifa_version = 22
        AND p.nationality_name = 'Brazil'
        AND t.league_name = 'Premier League'
    ORDER BY p.overall DESC
    LIMIT 10
"""

# Execute the query and display top 10 Brazilian players in Premier League for FIFA 22
spark.sql(query_brazilians).show(truncate=False)


+---------------+----------+-----------------+-------+
|Player         |Positions |Team             |overall|
+---------------+----------+-----------------+-------+
|Ederson        |GK        |Manchester City  |89     |
|Alisson        |GK        |Liverpool        |89     |
|Fabinho        |CDM, CB   |Liverpool        |86     |
|Thiago Silva   |CB        |Chelsea          |85     |
|Roberto Firmino|CF        |Liverpool        |85     |
|Gabriel Jesus  |ST        |Manchester City  |83     |
|Fernandinho    |CDM, CB   |Manchester City  |83     |
|Allan          |CDM, CM   |Everton          |83     |
|Richarlison    |ST, LW, LM|Everton          |82     |
|Alex Telles    |LB        |Manchester United|82     |
+---------------+----------+-----------------+-------+

